In [131]:
import overpass
import geopandas
import matplotlib.pyplot as plt
import folium
import geojson

def bbox_to_bounds(bbox):
    bo = bbox.split(", ")
    bounds = [[bo[0], bo[3]], [bo[2], bo[1]]]

    return bounds

def overpass_this(element, key, value = None):
    if value != None:
        q = '{}["{}"="{}"]({})'.format(element,key, value, bbox)
    else:
        q = '{}["{}"]({})'.format(element,key,bbox)
    resp = api.Get(q)
    name = "{element}-{key}-{value}.geojson".format(element=element, key=key, value=value)
    open(name, "w").write(str(resp))
    return geopandas.read_file(name)

def new_map(style='cartodbpositron'):
    m = folium.Map(tiles=style)
    m.fit_bounds(bounds)
    return m

def dump(data, mapview = None):
    if mapview == None:
        mapview = new_map()
    points = folium.features.GeoJson(geojson.dumps(data))
    mapview.add_children(points)
    return mapview

from IPython.display import HTML
def street_views(locations, height = 250):
    html_code = ''
    for location in locations:
        html_code = html_code + """
<iframe style="width:100%;height:{height}px;"
frameborder="0" scrolling="no" marginheight="0" marginwidth="0"
src="http://my.ctrlq.org/maps/#street|0.01|0|0|{y_coord}|{x_coord}">
</iframe>""".format(
           x_coord = location.centroid.x,
           y_coord = location.centroid.y,
           height = height
        )
    return HTML(html_code)

def earth_view(locations, height=400, zoom= 19):
    html_code= ''
    for location in locations:
        html_code = """<iframe
        style="width: 100%; height: {height}px;" frameborder="0" scrolling="no" marginheight="0" marginwidth="0"
        src="http://my.ctrlq.org/maps/#hybrid|{zoom}|{y_coord}|{x_coord}"></iframe>
        """.format(
           x_coord = location.centroid.x,
           y_coord = location.centroid.y,
           height = height,
           zoom=zoom,
        )
    return HTML(html_code)

import tempfile
def to_GeoSeries(geojson):
    temp = tempfile.mktemp(prefix="tmp_geo_", suffix=".geojson")
    open(temp, "w").write(str(geojson))
    return geopandas.read_file(temp)
    

bbox = "59.30516, 10.55511, 59.47508, 10.83115"
bounds = bbox_to_bounds(bbox)
api = overpass.API()


# Initialize bounding box and API for Moss

In [ ]:
bbox = "59.30516, 10.55511, 59.47508, 10.83115"
bounds = bbox_to_bounds(bbox)
api = overpass.API()

In [16]:
b = overpass_this("way", "bridge")
t = overpass_this("way", "tunnel")
m = new_map(style='Stamen Terrain')
dump(b, m)
dump(t, m)

# Open A streetmap view

This is not from any particular place. but it is nice to have the streetview in nteract, since it performs better than a browser, and saves from context-switching.

In [29]:
earth_view()

In [38]:
r = overpass_this("way", "junction", "roundabout")
n = new_map("OpenStreetMap")
r = r.buffer(0.001)
dump(r,n)
dump(t,n)

# Tunnels that are near roundabouts

In [39]:

intersection = r.unary_union.intersection(t.unary_union) 
dump(intersection)

In [40]:
street_views(intersection)

# Brute forcing through roundabouts

None of the ones near tunnel where the correct ones, so we spread the serach.

In [41]:
street_views(r)

# Lets look for hill

There is a topology dataset, but let's first see what openstreetmap has to offer.

In [53]:
h = overpass_this("node", "natural", "peak")
r = overpass_this("way", "junction", "roundabout")
r.buffer(0.001)
n = new_map()
dump(h,n)
dump(r,n)

In [43]:
h

,description:no,ele,geometry,name,natural
0,Moss kommunetopp.,140,POINT (10.6355352 59.4549791),Rambergåsen,peak


In [56]:
earth_view(h.geometry)



# Moving to Troms

The buildings look rather more typical for northern Norway, and there are now hills like the one in the background in Moss.

Let's start by lookuin up the most obvious things, the roundabouts.


In [144]:
bbox= "69.6191, 18.80104, 69.72358, 19.09836"
bounds = bbox_to_bounds(bbox)

In [143]:
tr = overpass_this("way", "junction", "roundabout")
tr = tr.buffer(0.005)
tr = tr.unary_union
trcs = geopandas.GeoSeries([poly.centroid for poly in tr])
tm = new_map()
dump(trcs,tm)
dump(tr, tm)

# A narrower filter

The idea was to narrow the seach area by removing elements that are not on the east side of a `"natural"="peak"`. However, it seems that the hills in Tromsø do not fill the Norwegian criteria for a peak :smile:

In [86]:
tr = overpass_this("way", "junction", "roundabout")
tt = overpass_this("way", "tunnel")

road_types = ["pedestrian", "footway"]
tsr = api.Get(
    """(
    way["highway"="footway"]({bbox});
    way["highway"="pedestrian"]({bbox})
)
    """.format(bbox = bbox))
tb = api.Get('way["bridge"]({})'.format(bbox))
# this qery returns data that will not be ofx use
#tp = overpass_this("node", "natural", "peak")
tmap = new_map("Stamen Terrain")
tr = tr.to_crs({'proj': "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"})
tr = tr.buffer(100)
tr = tr.to_crs({'init' :'epsg:4326'}) 
dump(tr, tmap)
dump(tsr, tmap)
#dump(tsr, tmap)


In [149]:
print(type(tr))
print(type(tsr))
tsrGs = to_GeoSeries(tsr)
print(type(tsrGs))

<class 'shapely.geometry.multipolygon.MultiPolygon'>
<class 'geojson.feature.FeatureCollection'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [153]:
tsrGsU = tsrGs.unary_union
pois = tru.intersection(tsrGsU)
print(type(pois))
m = new_map()
dump(pois,m)
dump(tru,m)

<class 'shapely.geometry.multilinestring.MultiLineString'>




# Mapillary photo keys


- red backside of round street mirror: kvlvqd7cUvDPi5xosLgoVQ
- yellow building in on street level in the background: Y6AzuKhr7qZ1K2WuOA33CA

![](https://d1cuyjsrcm0gby.cloudfront.net/kvlvqd7cUvDPi5xosLgoVQ/thumb-2048.jpg)
![](https://d1cuyjsrcm0gby.cloudfront.net/Y6AzuKhr7qZ1K2WuOA33CA/thumb-2048.jpg)

In [173]:
tr = overpass_this("way", "junction", "roundabout")
tsr = api.Get(
    """(
    way["highway"="footway"]({bbox});
    way["highway"="pedestrian"]({bbox})
    )
    """.format(bbox = bbox))

# roundtrip to utm33 for buffering
tr = tr.to_crs({'proj': "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"})
tr = tr.buffer(100)
tr = tr.to_crs({'init' :'epsg:4326'})
# ROIs in dataset intersections
tru = tr.unary_union
tsrGs = to_GeoSeries(tsr)
rois = tru.intersection(tsrGs.unary_union)


# query returns useless data
#tp = overpass_this("node", "natural", "peak")
tmap = new_map("Cartodb dark_matter")
dump(rois,tmap)
dump(tru,tmap)
tmap